In [ ]:
import import_ipynb
import Preparing_Data
import Learning_Models

%run ./Preparing_Data.ipynb
%run ./Learning_Models.ipynb

In [ ]:
# Preparing Data for Index Prediction

# Get the Index Info

df_index_info = {'Regime':True, 'Value':False, 'Fundamental':False, 'Oil':False, 'Risk_free':False, 'Dollar':True}
df_index, first_date, all_days = Create_Index(df_index_info)

# preprocessing Data
lags_return = [1,3,5,10,20]
df_index, columns = calculating_Macro_features(df_index = df_index, df_index_info = df_index_info, lags_return = lags_return)
list_info = df_index.columns.tolist()
list_info = [item for item in list_info if item not in columns['OUTPUT']]

train_test_split = int(np.floor(df_index.shape[0] * 0.8))
df_index_train = df_index.iloc[0:train_test_split,:]
df_index_test  = df_index.iloc[train_test_split:,:]

y_train_index = df_index_train[columns['OUTPUT']]
y_test_index  = df_index_test[columns['OUTPUT']]

X_train_index = df_index_train[list_info]
X_test_index  = df_index_test[list_info]

predictors = {'Trend':True, 'Value':True, 'Exchange':True, 'risk_free':False, 'Oil':True,'fundamental_value':False, 'fundamental_ratios':False, 'regime':True}
X_train_index, X_test_index = Select_Features_Categories_index(X_train = X_train_index, X_test = X_test_index, feat_dict = predictors, columns = columns)


In [ ]:
y_pred_index_binary, y_proba_index_binary, _, importance_index_binary, index_model_binary = XGB(X_train_index, X_test_index, y_train_index['binary_3_days'], y_test_index['binary_3_days'], mode = 'binary', is_binary = True)
y_pred_index_multi,  y_proba_index_multi, _,  importance_index_multi,  index_model_multi  = XGB(X_train_index, X_test_index, y_train_index['class_3_days'] - 1, y_test_index['class_3_days'] - 1, mode = 'multi', is_binary = False)

In [ ]:
#
# warnings.filterwarnings('ignore')
# for i in range(100,df_index.shape[0]):
#     if i == 100:
#         df_index = regime_detection(price = df_index, col = 'index_vwap') 
#     else:
#         print(i)
#         df = df_index[:i] 
#         df = regime_detection(price = df, col = 'index_vwap')
#         df_index['regime'].iloc[i] = df['regime'].iloc[-1]
# warnings.simplefilter('default')

In [ ]:
# Analyze the output

frame = 'Binary'

if frame == 'Mulit':
    Class = ['predicted']
    prediction = pd.DataFrame(y_pred_index_multi, columns=Class, index=y_test_index.index)
    prediction['happened'] = y_test_index['class_3_days']
    prediction['predicted'] = prediction['predicted'] + 1
    prediction['error'] = np.abs(prediction['happened'] - prediction['predicted'])
    prediction['return'] = y_test_index['ret_3_days']

if frame == 'Binary':
    Class = ['Class 1']
    prediction = pd.DataFrame(y_proba_index_binary, columns=Class, index=y_test_index.index)
    prediction['happened'] = y_test_index['class_3_days']
    prediction['return'] = y_test_index['ret_3_days']
    quantiles_by_date_Class1 = prediction['Class 1'].quantile([0, 0.25, 0.5, 0.75, 1])
    prediction['predicted'] = 0
    for i in range(4):
        prediction.loc[(prediction['Class 1'] >= quantiles_by_date_Class1.iloc[i]) & (prediction['Class 1'] <= quantiles_by_date_Class1.iloc[i+1]), 'predicted'] = i+1
    prediction['error'] = np.abs(prediction['happened'] - prediction['predicted'])


In [ ]:
# Print the Result

print('')

print('Average Absolute Error = ' + f"{prediction['error'].mean():.3f}")

print('')

print('return very_high is =   ' + f"{prediction.loc[prediction['predicted'] == 4, 'return'].sum() * 100:.2f}" + ' %')
print('return high is =        ' + f"{prediction.loc[prediction['predicted'] == 3, 'return'].sum() * 100:.2f}" + ' %')
print('return low is =         ' + f"{prediction.loc[prediction['predicted'] == 2, 'return'].sum() * 100:.2f}" + ' %')
print('return very_low is =    ' + f"{prediction.loc[prediction['predicted'] == 1, 'return'].sum() * 100:.2f}" + ' %')

a = [prediction.loc[prediction['predicted'] == 1, 'return'].sum() * 100, prediction.loc[prediction['predicted'] == 2, 'return'].sum() * 100, prediction.loc[prediction['predicted'] == 3, 'return'].sum() * 100,
    prediction.loc[prediction['predicted'] == 4, 'return'].sum() * 100] 

plt.figure(figsize=(8, 5))
plt.bar(range(len(a)), a, tick_label=range(1, len(a) + 1))
plt.xlabel('Class')
plt.ylabel('Average Daily Return %')
# plt.title('Multi-Class Classification Result')
plt.savefig('bar_chart.png')
plt.show()
plt.close()

In [ ]:
# Confusion Matrix

conf_matrix = confusion_matrix(prediction['happened'], prediction['predicted'], labels=[1, 2, 3, 4])
conf_matrix_percentage = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

# Plotting the confusion matrix with percentages
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_percentage, annot=True, fmt='.2f', cmap='Blues', xticklabels=[1, 2, 3, 4], yticklabels=[1, 2, 3, 4])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Percentages)')
plt.savefig('confusionMatrix.png')
plt.show()
